In [16]:
# Import the required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path

## Prepare the Hospital Claims DataFrame

In [17]:
# Read in hospital_claims.csv file as a Pandas DataFrame
hospital_data = pd.read_csv(Path("../Resources/hospital_claims.csv"))

# Review the DataFrame
hospital_data.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,32963.07,5777.24,4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,15131.85,5787.57,4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,37560.37,5434.95,4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,13998.28,5417.56,4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,31633.27,5658.33,4851.44


In [18]:
# Slice the DataFrame to consist of only "552 - MEDICAL BACK PROBLEMS W/O MCC" information
procedure_552_charges = hospital_data[
    hospital_data["DRG Definition"] == "552 - MEDICAL BACK PROBLEMS W/O MCC"
]

# Review the DataFrame
procedure_552_charges.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
118109,552 - MEDICAL BACK PROBLEMS W/O MCC,50128,TRI-CITY MEDICAL CENTER,4002 VISTA WAY,OCEANSIDE,CA,92056,CA - San Diego,14,26610.85,5970.57,5007.14
118137,552 - MEDICAL BACK PROBLEMS W/O MCC,50764,SHASTA REGIONAL MEDICAL CENTER,1100 BUTTE ST,REDDING,CA,96001,CA - Redding,29,36127.31,6305.17,5219.93
118355,552 - MEDICAL BACK PROBLEMS W/O MCC,60001,NORTH COLORADO MEDICAL CENTER,1801 16TH STREET,GREELEY,CO,80631,CO - Greeley,48,20543.02,5454.12,4252.91
118383,552 - MEDICAL BACK PROBLEMS W/O MCC,100009,UNIVERSITY OF MIAMI HOSPITAL,1400 NW 12TH AVE,MIAMI,FL,33136,FL - Miami,37,40265.86,6362.72,5466.67
120657,552 - MEDICAL BACK PROBLEMS W/O MCC,100012,LEE MEMORIAL HOSPITAL,2776 CLEVELAND AVE,FORT MYERS,FL,33901,FL - Fort Myers,83,20712.84,4889.45,3755.37


## Slice data for Average Total Payments by State

In [19]:
# Group data by state and average total payments, and then sum the values
payments_by_state = procedure_552_charges[["Average Total Payments", "Provider State"]]

# Sum the average total payments by state
total_payments_by_state = payments_by_state.groupby("Provider State").sum()

# Review the DataFrame
total_payments_by_state.head()

,Average Total Payments
Provider State,
AK,7134.40
AL,149613.29
AR,116147.95
AZ,151868.43
CA,1068146.26


### Create the plot objects

In [20]:
# Plot data using hvplot.bar
total_payments_by_state.hvplot.bar()

:Bars   [Provider State]   (Average Total Payments)

In [21]:
# Sort the state data values by Average Total Paymnts
sorted_total_payments_by_state = total_payments_by_state.sort_values("Average Total Payments")

# Plot the sorted data
sorted_total_payments_by_state.hvplot()

:Curve   [Provider State]   (Average Total Payments)

## Compose side-by-side plots using the + operator

In [22]:
# Compose plots using + operator.  
total_payments_by_state.hvplot.bar() + sorted_total_payments_by_state.hvplot()

:Layout
   .Bars.I  :Bars   [Provider State]   (Average Total Payments)
   .Curve.I :Curve   [Provider State]   (Average Total Payments)

In [23]:
# We prevent this by renaming the index in one of the dataframes before plotting them.
sorted_total_payments_by_state.index.names = ['Provider State Sorted']

total_payments_by_state.hvplot.bar() + sorted_total_payments_by_state.hvplot()

:Layout
   .Bars.I  :Bars   [Provider State]   (Average Total Payments)
   .Curve.I :Curve   [Provider State Sorted]   (Average Total Payments)

## Slice data for Average Medicare Payments by State

In [24]:
# Group data by state and average medicare payments, and then sum the values
medicare_payment_by_state = procedure_552_charges[["Average Medicare Payments", "Provider State"]]

total_medicare_by_state = medicare_payment_by_state.groupby("Provider State").sum()

total_medicare_by_state.head()

,Average Medicare Payments
Provider State,
AK,6008.80
AL,112154.39
AR,90176.26
AZ,120926.12
CA,901778.32


In [25]:
# Sort data values
sorted_total_medicare_by_state = total_medicare_by_state.sort_values("Average Medicare Payments")

sorted_total_medicare_by_state.hvplot.bar()

:Bars   [Provider State]   (Average Medicare Payments)

## Compose overlay plots using the + operator

In [30]:
# Overlay plots of different type using * operator
sorted_total_payments_by_state.hvplot.line(height = 300, width = 1000, rot = 90, label="Average Total Payments") * sorted_total_medicare_by_state.hvplot.bar(height = 300, width = 1000, rot = 90, label="Average Medicare Payments")

:Overlay
   .Curve.Average_Total_Payments   :Curve   [Provider State Sorted]   (Average Total Payments)
   .Bars.Average_Medicare_Payments :Bars   [Provider State]   (Average Medicare Payments)

In [29]:
# Overlay plots of the same type using * operator
sorted_total_payments_by_state.hvplot.bar(height = 300, width = 1000, rot = 90, label="Average Total Payments") * sorted_total_medicare_by_state.hvplot.bar(height = 300, width = 1000, rot = 90, label="Average Medicare Payments")

:Overlay
   .Bars.Average_Total_Payments    :Bars   [Provider State Sorted]   (Average Total Payments)
   .Bars.Average_Medicare_Payments :Bars   [Provider State]   (Average Medicare Payments)